# Finetuning a pretrained YOLO detection model with segmentation data:

## Imports:

In [1]:
import os
import datetime
import tensorflow as tf
!apt-get install pv


## Fetch the data:

In [7]:
if 'data_loaded' not in globals():
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir /content/Data
    !cp -rv /content/drive/MyDrive/setup_015/* /content/Data/ | pv
    !cp -v /content/drive/MyDrive/setup-002-standalone.yaml /content/Data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/MyDrive/setup_015/001.png' -> '/content/Data/001.png'
'/content/drive/MyDrive/setup_015/002.png' -> '/content/Data/002.png'
'/content/drive/MyDrive/setup_015/003.png' -> '/content/Data/003.png'
'/content/drive/MyDrive/setup_015/004.png' -> '/content/Data/004.png'
'/content/drive/MyDrive/setup_015/005.png' -> '/content/Data/005.png'
'/content/drive/MyDrive/setup_015/006.png' -> '/content/Data/006.png'
'/content/drive/MyDrive/setup_015/007.png' -> '/content/Data/007.png'
'/content/drive/MyDrive/setup_015/008.png' -> '/content/Data/008.png'
'/content/drive/MyDrive/setup_015/009.png' -> '/content/Data/009.png'
'/content/drive/MyDrive/setup_015/010.png' -> '/content/Data/010.png'
'/content/drive/MyDrive/setup_015/011.png' -> '/content/Data/011.png'
'/content/drive/MyDrive/setup_015/012.png' -> '/content/Data/012.png'
'/content/drive/MyDrive/setup_0

## Fetch helper scripts/functions and restructure the data:

In [9]:
if 'data_loaded' not in globals():
    !cp -v /content/drive/MyDrive/prepare_yolo_data_folders.py /content/
    from prepare_yolo_data_folders import prepare_yolo_data_folders

    # Colab magic to make "Data" the current directory:
    %cd /content/Data

    #Do the train-validation split and move the image and label files to YOLO folder structure:
    data_loaded = prepare_yolo_data_folders()

'/content/drive/MyDrive/prepare_yolo_data_folders.py' -> '/content/prepare_yolo_data_folders.py'
/content/Data
Current folder is: /content/Data
Found 66 labels.
Found 66 labelled images:
 ['011', '018', '013', '034', '004', '048', '003', '009', '020', '065', '025', '068', '022', '030', '055', '024', '032', '047', '043', '042', '061', '031', '038', '053', '041', '060', '064', '069', '050', '037', '046', '028', '023', '054', '040', '021', '066', '052', '012', '044', '067', '062', '026', '049', '027', '016', '051', '015', '063', '056', '029', '033', '058', '019', '039', '035', '059', '036', '017', '010', '005', '014', '006', '057', '001', '002']
Created train/images and train/labels.
Created val/images and val/labels.
Created test/images and test/labels.
Moved 011.png to train/images.
Moved converted_segmentation_labels/011.txt to train/labels.
Moved 018.png to train/images.
Moved converted_segmentation_labels/018.txt to train/labels.
Moved 013.png to train/images.
Moved converted_segment

## Install ultralytics and import YOLO:

In [10]:
%pip -q install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.8/874.8 kB 26.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Okay, so the images and labels are there? Next: Import YOLO model for training!
Note: Check Bboxes vs. segmentations!

## Load tensorboard:

In [11]:
%load_ext tensorboard


## Import YOLO model for training

In [18]:
# Load a pretrained model:
model = YOLO("yolov8m-seg.pt")

# Other models:
#model = YOLO("yolov8s.pt")
#model = YOLO("yolov8m.pt")
#model = YOLO("yolov8n.pt")

### YAML file for folder and class definition: Everything ok?

In [13]:
#!cat /content/misc/setup-002-standalone.yaml
yamlfile = "/content/Data/setup-002-standalone.yaml"

## Do the training!

In [19]:
# If GPU-RAM is full before training:

# import torch
# torch.cuda.empty_cache()


In [23]:
# Train the model
#trainresults = model.train(data="/content/setup-002-standalone.yaml", epochs=100, imgsz=1280)
trainresults = model.train(data=yamlfile, epochs=100, imgsz=1280, batch=0.9)

Ultralytics YOLOv8.2.102 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/content/Data/setup-002-standalone.yaml, epochs=100, time=None, patience=100, batch=0.9, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4222, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

train: Scanning /content/Data/train/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Data/val/labels.cache... 13 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<?, ?it/s]


Plotting labels to runs/segment/train4222/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 86 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to runs/segment/train4222
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      8.26G      1.575      1.724      1.102      2.044         20       1280: 100%|██████████| 12/12 [00:04<00:00,  2.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.40it/s]

                   all         13         98      0.636      0.663      0.676      0.328      0.212     0.0408     0.0165    0.00321



100 epochs completed in 0.178 hours.
Optimizer stripped from runs/segment/train4222/weights/last.pt, 54.9MB
Optimizer stripped from runs/segment/train4222/weights/best.pt, 54.9MB

Validating runs/segment/train4222/weights/best.pt...
Ultralytics YOLOv8.2.102 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8m-seg summary (fused): 245 layers, 27,222,963 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.40it/s]


                   all         13         98      0.616      0.694      0.678       0.34      0.197     0.0306      0.019    0.00411
Speed: 0.5ms preprocess, 18.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/segment/train4222


### wandb?
TODO: think about saving the models automatically ...

... and managing the parameter "matrix"

### Tensorboard:

In [ ]:
# Show tensorboard:
#%tensorboard --logdir /content/runs/detect #path/to/runs

## Predict the cucumbers on test images:

In [ ]:
# Examples:
# #predictresults = model(["image1.jpg", "image2.jpg"])  # return a list of Results objects
# # Run inference on 'bus.jpg' with arguments
# model.predict("bus.jpg", save=True, imgsz=320, conf=0.5)


# /content/Data/test/images

# First, I guess we should _dynamically_?! define a list of images to predict on:
#imagelist = []
imagelist = os.listdir("/content/Data/test/images")
print(f"Using {len(imagelist)} images for prediction")

# Then, we can predict on the images:
results = model.predict(imagelist,
                        imgsz=1280,
                        conf=0.5,
                        #show=True,
                        save=True)

print("Where are the resulting pictures stored? :-) \n" +
      "And how can I rescale them for/after prediction?")

# # Process result:
# # for result in results:
# boxes = result.boxes  # Boxes object for bounding box outputs
# # masks = result.masks  # Masks object for segmentation masks outputs
# # keypoints = result.keypoints  # Keypoints object for pose outputs
# probs = result.probs  # Probs object for classification outputs
# obb = result.obb  # Oriented boxes object for OBB outputs
# result.show()  # display to screen

### Show resulting pictures:

Perhaps replace IPython display by PIL stuff?

In [ ]:
from IPython.display import Image
#Image(filename='/content/runs/detect/train53/20220614_134218_Gurkenfoto_2.jpg', width=400)


In [ ]:
#Image(filename='/content/runs/detect/train2/20220614_134218_Gurkenfoto_rescaled_640.jpg', width=400)


In [ ]:
#Image(filename='/content/runs/detect/train53/dsc01843.png', width=400)